# Initalize libaries and file name
## !!!(YOU NEED TO PIP INSTALL h5py)!!! "conda install h5py" in console

In [ ]:
file_name = 'ACa-01_25x1.xyz'

# These need to be defined based on resolution, and hence and predefined before running the rest of the code
Xres = 3.484 #um
Yres = Xres #um

# Set font size here
fsize = 20

# --------------------------------------------------------------------------------------------------------------------------------------------------------

#Grab those libaries, I ain't writing shit

from mpl_toolkits import mplot3d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set global font to Times New Roman and font size to 10pt
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 10

import random
from statistics import mean
from matplotlib import cm
from matplotlib.ticker import MaxNLocator
#from matplotlib.mlab import griddata
from scipy import interpolate
import h5py

import scipy
from scipy import stats
import math
import time

# Create the output file name based on the input file name
output_file_name = file_name.replace('.xyz', '_piv.xyz')

# Replace the file extension for the output CSV file
saving_file_name = file_name.replace('.xyz', '_processed.h5')

# This is the conversion of the Zygo data to format the code wants

In [ ]:
start_time = time.time()

# Read the CSV file, treating multiple spaces as a single delimiter
df = pd.read_csv(file_name, sep='\\s+',skiprows=14)
print('Read the CSV file')

# Drop the last column
df = df.iloc[:, :-1]
print('Dropped the last column')

# Replace "No" with NaN
df.replace("No", np.nan, inplace=True)
df = df.iloc[:-1]
print('Replaced "No" with NaN and dropped the last row')

# Assuming the DataFrame has the structure [X, Y, Z]
df.columns = ['X', 'Y', 'Z']  # Assign column names if needed
df['X'] = df['X'].astype('int64')
df['Y'] = df['Y'].astype('int64')
print(df)

# Pivot the DataFrame to create the 2D map
pivot_table = df.pivot(index='Y', columns='X', values='Z')
print('Pivoted the DataFrame')

# Save the cleaned and pivoted DataFrame to the new file
pivot_table.to_csv(output_file_name, index=0, header=1)
print(f"Saved processed data to {output_file_name}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Function execution time: {elapsed_time:.2f} seconds")

# Main body of the code, does missing data interpolation and tilt correction

In [ ]:
start = time.time()

#Step 1: read the data
#------------#------------#------------#------------#------------#------------#------------#------------

A = pd.read_csv(output_file_name)


edge = 0.2 #distance (mm) from sample edges to be ignored during tilt correction
movingaverage = 15
stdrange = 1 #Number of standard deviations from mean which is accepted
movav = movingaverage - 1
print('Edge size / mm: ', edge)

#This grabs all the real data from our now 2D array
B = A.iloc[:,1:]


#Step 2: Find the mid point
#------------#------------#------------#------------#------------#------------#------------#------------

#Important MINIUM POINT HERE, NOT MID POINT
minpoints = 250 #Minimum number of data points in a column that we'll consider as being a valid number 

#Finding the centre of the sample
for i in np.linspace(0, (B.shape[1]-1), (B.shape[1])):#[row, col]
    Bslice = B.iloc[:,int(i)] #slice for column
    Bfloat = Bslice.astype(np.float64) #float64 datatype necessary for np.isnan command to work
    if np.isnan(Bfloat. values).sum()<B.shape[0]-(minpoints-1): #i.e. at least 10 data points
        leftedge = i
        break
    else :
        ()
        
for i in np.linspace((B.shape[1]-1), 0, (B.shape[1])):#[row, col]
    Bslice = B.iloc[:,int(i)] #slice for column
    Bfloat = Bslice.astype(np.float64) #float64 datatype necessary for np.isnan command to work
    if np.isnan(Bfloat. values).sum()<B.shape[0]-(minpoints-1): #i.e. at least 10 data points
        rightedge = i
        break
    else :
        ()
        
        
for i in np.linspace(0, (B.shape[0]-1), (B.shape[0])):#[row, col]
    Bslice = B.iloc[int(i), :] #slice for column
    Bfloat = Bslice.astype(np.float64) #float64 datatype necessary for np.isnan command to work
    if np.isnan(Bfloat. values).sum()<B.shape[1]-(minpoints-1): #i.e. at least 10 data points
        topedge = i
        break
    else :
        ()
        
for i in np.linspace((B.shape[0]-1), 0, (B.shape[0])):#[row, col]
    Bslice = B.iloc[int(i), :] #slice for column
    Bfloat = Bslice.astype(np.float64) #float64 datatype necessary for np.isnan command to work
    if np.isnan(Bfloat. values).sum()<B.shape[1]-(minpoints-1): #i.e. at least 10 data points
        bottomedge = i
        break
    else :
        ()
        
        
        
ymid = mean((leftedge, rightedge)) #since the dataset will be transposed
xmid = mean((topedge, bottomedge)) #since the dataset will be transposed


#Transpose to match physical specimen
dfraw1 = (pd.DataFrame(data=B).T).astype(float)
dfraw = pd.DataFrame(data=dfraw1)
dfnew = dfraw*1


#Define sample centres: X & Y apparently swapped due to dataframe being transposed after xmid and ymid were found
Y0 = xmid
X0 = ymid

print('Left Edge column number: ', leftedge)
print('Right Edge column number: ', rightedge)
print('Top Edge column number: ', topedge)
print('Bottom Edge column number: ', bottomedge)
print('X0: ', X0)
print('Y0: ', Y0)
print('-----------------------------------------')

#Define X & Y axes from dataset

# #trying to add px back in
# px = Xres*Yres
# #DANGER DANGER DOM ADDED THIS CODE

X = (np.linspace(0, dfraw.shape[1]-1, dfraw.shape[1])-X0)*Xres/1000 # 1px = 2.554862 um

Y = (np.linspace(0, dfraw.shape[0]-1, dfraw.shape[0])-Y0)*Yres/1000 # 1px = 2.558354 um


#Step 3: Cut off the edges of the data after filtering
#------------#------------#------------#------------#------------#------------#------------#------------

#New, simpler edge removal 5/2/2021 - trying it before filtering 22/2/2021
#Removing Y-edges
Yedge = 1*dfraw


#Construction of outer and inner radii to remove edge effects
meanres = (Xres+Yres)/2000 #in mm, different resolution in X & Y
rout = (6.35 - edge)/meanres #in px
rin = (3.1875 + edge)/meanres


#Outside of left outer edge (i.e. i<Xoutneg)
for i in range(0, int(X0-rout)+1, 1):
    Yedge.iloc[i,:] = np.nan

#Outside of right outer edge (i.e. i>Xoutneg)
for i in range(int(X0+rout)+1, dfraw.shape[0], 1):
    Yedge.iloc[i,:] = np.nan

#Outside of lower outer edge (i.e. i<Youtneg)
for i in range(0, int(Y0-rout)+1, 1):
    Yedge.iloc[:, i] = np.nan

#Outside of upper outer edge (i.e. i>Youtneg)
for i in range(int(Y0+rout)+1, dfraw.shape[1], 1):
    Yedge.iloc[:, i] = np.nan
     
    
#Between Xout and Xin (for both sides)
for i in range(int(X0-rout)+1, int(X0)+1, 1):
    Youtpos = int(Y0 + math.sqrt(rout**2 - (i-X0)**2))
    Youtneg = int(Y0 - math.sqrt(rout**2 - (i-X0)**2))
    
    Yedge.iloc[i, 0:Youtneg] = np.nan
    Yedge.iloc[i, Youtpos:Yedge.shape[1]-1] = np.nan

for i in range(int(X0), int(X0+rout), 1):
    Youtpos = int(Y0 + math.sqrt(rout**2 - (i-X0)**2))
    Youtneg = int(Y0 - math.sqrt(rout**2 - (i-X0)**2))

    Yedge.iloc[i, 0:Youtneg] = np.nan
    Yedge.iloc[i, Youtpos:Yedge.shape[1]-1] = np.nan

#Between Xinneg and Xinpos (hole in middle as a single section)
for i in range(int(X0-rin)+1, int(X0+rin)-1, 1):
    #Outer portion
    Youtpos = int(Y0 + math.sqrt(rout**2 - (i-X0)**2))
    Youtneg = int(Y0 - math.sqrt(rout**2 - (i-X0)**2))
    
    Yedge.iloc[i,0:Youtneg] = np.nan
    Yedge.iloc[i, Youtpos:Yedge.shape[1]-1] = np.nan
        
    #Inner portion
    Yinpos = int(Y0 + math.sqrt(rin**2 - (i-X0)**2))
    Yinneg = int(Y0 - math.sqrt(rin**2 - (i-X0)**2))
    
    Yedge.iloc[i, Yinneg:Yinpos] = np.nan

dfi = Yedge


#Step 4: Remove the peaks and troughs for plane tilt analysis
#------------#------------#------------#------------#------------#------------#------------#------------

#Filtering of data
#Excluding data outside of mean+x*standarddeviations, where x = std (below) and the mean is found using a moving average
print('Moving Point Average / - : ', movingaverage)
print('Number of Standard Deviations from Mean Allowed: ', stdrange)




dfiflatX = np.ndarray.flatten(np.asarray(dfi))
meanstdflatX = 1*dfiflatX
mstdfl = 1*dfiflatX  #Need 1* so that the 'rawflat' array remains unchanged (not sure why it was changing)
mstdflX = 1*meanstdflatX

dfiflatY = np.ndarray.flatten(np.asarray(dfi.T))
meanstdflatY = 1*dfiflatY
mstdflY = 1*meanstdflatY


srsX = pd.Series(mstdflX)
srsY = pd.Series(mstdflY)


rmX = srsX.rolling(movav+1, min_periods=1).mean()
rsX = srsX.rolling(movav+1, min_periods=1).std()

rmY = srsY.rolling(movav+1, min_periods=1).mean()
rsY = srsY.rolling(movav+1, min_periods=1).std()

        
for i in np.linspace(int(1 + (movav/2)), (len(dfiflatX)-1-(movav/2)), len(dfiflatX)-1-movav):
    if abs(rmX[int(i+movav/2)] - mstdflX[int(i)]) >= abs(stdrange*rsX[int(i+movav/2)]):
        mstdflX[int(i)] = np.nan
    elif round(mstdfl[int(i)],5)==round(rmX[int(i+movav/2)],5):
        mstdflX[int(i)] = np.nan
    else :
        ()

finish1 = time.time()
print('Intermediate time (s): ', finish1-start)
        
for i in np.linspace(int(1 + (movav/2)), (len(dfiflatX)-1-(movav/2)), len(dfiflatX)-1-movav):
    if abs(rmY[int(i+movav/2)] - mstdflY[int(i)]) >= abs(stdrange*rsY[int(i+movav/2)]):
        mstdflY[int(i)] = np.nan
    elif round(mstdfl[int(i)],5)==round(rmY[int(i+movav/2)],5):
        mstdflY[int(i)] = np.nan
    else :
        ()

        
rollY = pd.DataFrame(data=np.reshape(mstdflY, (dfi.shape[1], dfi.shape[0])))
rollYflat = np.ndarray.flatten(np.asarray(rollY.T))

for i in np.linspace(int(1 + (movav/2)), (len(dfiflatX)-1-(movav/2)), len(dfiflatX)-1-movav):
    if np.isnan(mstdflX[int(i)])==True or np.isnan(rollYflat[int(i)])==True:
        mstdfl[int(i)]=np.nan
    else :
        ()


        
#Returning the flattened array to a 2D dataframe (after reshaping into a 2D array)
dfedges = pd.DataFrame(data=np.reshape(mstdfl, (dfi.shape[0], dfi.shape[1])))
#dfedges.to_csv('11.1post.movav15std30or.csv', na_rep='NaN')

#Removing edges for analyses (written 20/8/2018 - notes in Book 2 p109, rewritten 04/10/2018 - notes in Book 2 p135)
print('Edges Max height / um: ', np.amax(dfedges.max()))
print('Edges Min height / um: ', np.amin(dfedges.min()))
print('Edges Rt / um', np.amax(dfedges.max())-np.amin(dfedges.min()))
print('-----------------------------------------')



finish2 = time.time()
print('Total Filtering Time (s): ', finish2-start)


#Step 5: Interpolate small to large in XY direction
#------------#------------#------------#------------#------------#------------#------------#------------

#Interpolate across whole sample (without edges)
dfint1 = dfedges.interpolate(method='linear', axis=1, limit=15)
dfint2 = dfint1.interpolate(method='linear', axis=0, limit=15)
dfint3 = dfint2.interpolate(method='linear', axis=1, limit=30)
dfint4 = dfint3.interpolate(method='linear', axis=0, limit=30)
dfint5 = dfint4.interpolate(method='linear', axis=1, limit=60)
dfint6 = dfint5.interpolate(method='linear', axis=0, limit=60)
dfint7 = dfint6.interpolate(method='linear', axis=1, limit=120)
dfint8 = dfint7.interpolate(method='linear', axis=0, limit=120)
dfint9 = dfint8.interpolate(method='linear', axis=1, limit=250)
dfint = dfint9.interpolate(method='linear', axis=0, limit=250) 

#Step 5.5: Re-remove edge filled in due to interpolation
#------------#------------#------------#------------#------------#------------#------------#------------

#Second edge removal
Yedge = 1*dfint

#Outside of left outer edge (i.e. i<Xoutneg)
for i in range(0, int(X0-rout)+1, 1):
    Yedge.iloc[i,:] = np.nan

#Outside of right outer edge (i.e. i>Xoutneg)
for i in range(int(X0+rout)+1, dfraw.shape[0], 1):
    Yedge.iloc[i,:] = np.nan

    
#Outside of lower outer edge (i.e. i<Youtneg)
for i in range(0, int(Y0-rout)+1, 1):
    Yedge.iloc[:, i] = np.nan

#Outside of upper outer edge (i.e. i>Youtneg)
for i in range(int(Y0+rout)+1, dfraw.shape[1], 1):
    Yedge.iloc[:, i] = np.nan
    
    
    
    
#Between Xout and Xin (for both sides)
for i in range(int(X0-rout)+1, int(X0)+1, 1):
    Youtpos = int(Y0 + math.sqrt(rout**2 - (i-X0)**2))
    Youtneg = int(Y0 - math.sqrt(rout**2 - (i-X0)**2))
    
    Yedge.iloc[i, 0:Youtneg] = np.nan
    Yedge.iloc[i, Youtpos:Yedge.shape[1]-1] = np.nan

for i in range(int(X0), int(X0+rout), 1):
    Youtpos = int(Y0 + math.sqrt(rout**2 - (i-X0)**2))
    Youtneg = int(Y0 - math.sqrt(rout**2 - (i-X0)**2))

    Yedge.iloc[i, 0:Youtneg] = np.nan
    Yedge.iloc[i, Youtpos:Yedge.shape[1]-1] = np.nan

#Between Xinneg and Xinpos (hole in middle as a single section)
for i in range(int(X0-rin)+1, int(X0+rin)-1, 1):
    #Outer portion
    Youtpos = int(Y0 + math.sqrt(rout**2 - (i-X0)**2))
    Youtneg = int(Y0 - math.sqrt(rout**2 - (i-X0)**2))
    
    Yedge.iloc[i,0:Youtneg] = np.nan
    Yedge.iloc[i, Youtpos:Yedge.shape[1]-1] = np.nan
        
    #Inner portion
    Yinpos = int(Y0 + math.sqrt(rin**2 - (i-X0)**2))
    Yinneg = int(Y0 - math.sqrt(rin**2 - (i-X0)**2))
    
    Yedge.iloc[i, Yinneg:Yinpos] = np.nan

ELI = Yedge #Where ELI stands for 'edgeless interpolated'



finish3 = time.time()
print('Interpolation Time (s): ', finish3-start)


#Step 6: Tilt correction
#------------#------------#------------#------------#------------#------------#------------#------------


#Tilt Correction
F = 1*ELI
eliflat = np.ndarray.flatten(np.asarray(ELI))
elimean = np.nanmean(eliflat)
elistd = np.nanstd(eliflat)

F[(F>elimean+elistd)]= np.nan
F[(F<elimean-elistd)] = np.nan

G = 1*F
fflat = np.ndarray.flatten(np.asarray(F))
fmean = np.nanmean(fflat)
fstd = 2*np.nanstd(fflat)

G[(G>fmean+fstd)]= np.nan
G[(G<fmean-fstd)] = np.nan
Gflat = np.ndarray.flatten(np.asarray(G))


Xrep = np.tile(X,len(Y))
Yrep = np.repeat(Y, len(X))

#Finding Xslope
LRX = pd.DataFrame({'Xrep': Xrep,'Gflat': Gflat})
LRXclean = LRX.dropna(axis=0)
slope, intercept = np.polyfit(LRXclean['Xrep'], LRXclean['Gflat'], 1)

#Correcting Xtilt
Xcorr = slope*X
ELIX = ELI.subtract(Xcorr, axis='columns')

#Define XTilt using correction factor
XTilt = math.cos(np.arctan(np.nanmean(slope)/1000))

#Finding Yslope
LRY = pd.DataFrame({'Yrep': Yrep,'Gflat': Gflat})
LRYclean = LRY.dropna(axis=0)
slope, intercept = np.polyfit(LRYclean['Yrep'], LRYclean['Gflat'], 1)

#Correcting Ytilt
Ycorr = slope*Y
ELIYX = ELIX.subtract(Ycorr, axis='rows')

#Define YTilt using correction factor
YTilt = math.cos(np.arctan(np.nanmean(slope)/1000))

#Define new X & Y positions
Xnew1 = X/XTilt
Ynew1 = Y/YTilt



finish4 = time.time()
print('Time for first tilt correction (s): ', finish4-start)


#Step 6.5: Do it again Trust me bro
#------------#------------#------------#------------#------------#------------#------------#------------

#Tilt Correction 2
H = 1*ELIYX
eliyxflat = np.ndarray.flatten(np.asarray(ELIYX))
eliyxmean = np.nanmean(eliyxflat)
eliyxstd = np.nanstd(eliyxflat)

H[(H>eliyxmean+eliyxstd)]= np.nan
H[(H<eliyxmean-eliyxstd)] = np.nan

I = 1*H
hflat = np.ndarray.flatten(np.asarray(H))
hmean = np.nanmean(hflat)
hstd = 2*np.nanstd(hflat)

I[(I>hmean+hstd)]= np.nan
I[(I<hmean-hstd)] = np.nan
Iflat = np.ndarray.flatten(np.asarray(I))


#Finding Xslope2
LRX2 = pd.DataFrame({'Xrep': Xrep,'Iflat': Iflat})
LRX2clean = LRX2.dropna(axis=0)
slope, intercept = np.polyfit(LRX2clean['Xrep'], LRX2clean['Iflat'], 1)

#Correcting Xtilt2
Xcorr2 = slope*X
ELIX2 = ELIYX.subtract(Xcorr2, axis='columns')

#Define XTilt2 using correction factor
XTilt2 = math.cos(np.arctan(np.nanmean(slope)/1000))


#Finding Yslope2
LRY2 = pd.DataFrame({'Yrep': Yrep,'Iflat': Iflat})
LRY2clean = LRY2.dropna(axis=0)
slope, intercept = np.polyfit(LRY2clean['Yrep'], LRY2clean['Iflat'], 1)

#Correcting Ytilt2
Ycorr2 = slope*Y
ELIYX2 = ELIX2.subtract(Ycorr2, axis='rows')

#Define YTilt2 using correction factor
YTilt2 = math.cos(np.arctan(np.nanmean(slope)/1000))

#Define new X & Y positions
Xnew = Xnew1/XTilt2
Ynew = Ynew1/YTilt2

finish5 = time.time()
print('Time for second tilt correction (s): ', finish5-start)


#Step 7: Translating so that the most populus plane is at zero
#------------#------------#------------#------------#------------#------------#------------#------------

#Initial translation so all data >=0, which makes finding the modal bin easier
ELIYXmin = np.amin(ELIYX2.min()) #Getting the value of ELImin
ELIYXT = ELIYX-ELIYXmin #Translating so minimimum is at Z=0 

#Fill in NaN cells as 0 (otherwise binning/histogram doesn't work)
ELIYXT0 = ELIYXT.fillna(0)

#Z-Correction: Finding the zero-plane and translating this
intervalbins = 2
A = np.ndarray.flatten(np.asarray(ELIYXT0))
Amax = int(np.amax(A))
#Set bins. Don't want to start at 0 since this will now contain what were previously NaN elements (ALL of them!) 
bins = list(range(intervalbins, Amax + intervalbins, intervalbins))
x, y, _ = plt.hist(A, bins) #Need dataframe to be a 1D array (NOT 2D!!!)
plt.title('Histogram showing frequency at Z-position')
plt.xlabel('Z Position / um')
plt.ylabel('Frequency / count')


#Finding POSITION of modal bin i.e. where should zero-plane be
#NB: np.argmax(x) finds number of bin (NB: starting at bin number 0) that max population is in
#print('Position of modal bin (counts): ', intervalbins*np.argmax(x), '-', intervalbins*(np.argmax(x)+1))
#print('Z-correction factor: ', intervalbins*(np.argmax(x)+0.5))


#Correcting Z such that the modal plane is at zero
ELIYXTT = ELIYXT - intervalbins*(np.argmax(x)+0.5)


print('------------------------')
print('Corrected Edgeless Max height / um: ', np.amax(ELIYXTT.max()))
print('Corrected Edgless Min height / um: ', np.amin(ELIYXTT.min()))
print('Corrected Edgless Rt / um: ', np.amax(ELIYXTT.max())-np.amin(ELIYXTT.min()))


#Step 8: Quantify data
#------------#------------#------------#------------#------------#------------#------------#------------

#Finding Sq and Ssk (quantification of data skew)

Zsortnan = np.sort(np.ndarray.flatten(np.asarray(ELIYXTT))) #Flattening the dataframe into a 1D array & ordering (doesn't matter but easy to do)
Zsort = Zsortnan[~np.isnan(Zsortnan)] #Removing the NaN cells (enabling further operations to be performed), -ve sign to remove NaN
Z2 = Zsort**2
Z3 = Zsort**3




px = 2.554862

#Finding the area of the data shown (i.e. not NaN), named 'Reduced Area' (RArea for short)
#Size of each pixel for corrected sample
Xnewsize = px/(1000*XTilt*XTilt2)
Ynewsize = px/(1000*YTilt*YTilt2)
RArea = (1000*Xnewsize)*(1000*Ynewsize)*len(Zsort) #Multiplied by 1000 (twice) since Z is in um andd X/Ynewsize are in mm

Sq = np.sqrt(np.sum(Z2)/RArea)
print('Sq / - :', Sq)

Ssk = (np.sum(Z3)/RArea)/(Sq**3)
print('Ssk / um:', Ssk)





# Create a figure with two subplots
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the original data on the first subplot
c1 = ax[0].contourf(X, Y, dfedges, 256, vmax=np.amax(dfedges), vmin=np.amin(dfedges))
ax[0].set_title('Original Data')
ax[0].axis('equal')
fig.colorbar(c1, ax=ax[0], ticks=range(-800, -100, 100), label='Z Position / um')
c1.set_clim(vmax=np.amax(dfedges), vmin=np.amin(dfedges))
ax[0].set_xlabel('X Position / mm')
ax[0].set_ylabel('Y Position / mm')
ax[0].spines["top"].set_visible(False)
ax[0].spines["right"].set_visible(False)

# Plot the corrected data on the second subplot
c2 = ax[1].contourf(Xnew, Ynew, ELIYXTT, 256, vmax=np.amax(ELIYXTT), vmin=np.amin(ELIYXTT))
ax[1].set_title('Corrected Data')
ax[1].axis('equal')
fig.colorbar(c2, ax=ax[1], ticks=range(-500, 500, 50), label='Z Position / um')
c2.set_clim(vmax=np.amax(ELIYXTT), vmin=np.amin(ELIYXTT))
ax[1].set_xlabel('X Position / mm')
ax[1].set_ylabel('Y Position / mm')
ax[1].spines["top"].set_visible(False)
ax[1].spines["right"].set_visible(False)

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0) # Spread out graphs to avoid overlap

# Resizing the plot area
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12 # X-axis size
fig_size[1] = 12 # Y-axis size

plt.show()

# Generate z_clean here so its not regenerated in other parts of the code
z_clean = ELIYXTT.to_numpy().flatten()[~np.isnan(ELIYXTT.to_numpy().flatten())]

print('Finished!')
end = time.time()
print('Time elapsed: ', end - start, ' seconds')

# This saves and loads the data into a HDF5 file so it won't have to be processed again

#### Saving

In [ ]:
# Xnew_grid, Ynew_grid = np.meshgrid(Xnew, Ynew)

# pixel size given in mm^2
pixel_size = ((Xres*1e-3) / (XTilt*XTilt2)) * ((Yres*1e-3) / (YTilt*YTilt2))
print(f'Pixel size is {pixel_size:.4g} mm^2')

# Save data to HDF5 file
with h5py.File(saving_file_name, 'w') as f:
    f.create_dataset('Xnew', data=Xnew)
    f.create_dataset('Ynew', data=Ynew)
    f.create_dataset('ELIYXTT', data=ELIYXTT)
    f.create_dataset('dfnew', data=dfnew)
    f.create_dataset('ELIX2', data=ELIX2)
    f.attrs['pixel_size'] = pixel_size

#### Loading

In [ ]:
# Load data from HDF5 file
with h5py.File(saving_file_name, 'r') as f:
    Xnew = f['Xnew'][:]
    Ynew = f['Ynew'][:]
    pixel_size = f.attrs['pixel_size']
    ELIYXTT = pd.DataFrame(f['ELIYXTT'][:])
    dfnew = pd.DataFrame(f['dfnew'][:])
    ELIX2 = pd.DataFrame(f['ELIX2'][:])

# Generate z_clean again in case processing code was not run
z_clean = ELIYXTT.to_numpy().flatten()[~np.isnan(ELIYXTT.to_numpy().flatten())]

#### This is so dumb but look at it (3D spinning gif)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter
import time

# Start timing
start_time = time.time()

# Set figure size to 1920x1080 resolution with 300 DPI
fig = plt.figure(figsize=(19.2, 10.8), dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Parameters
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, 2 * np.pi, 50)
u, v = np.meshgrid(u, v)

# Create the 3D contour plot
contour = ax.contour3D(Xnew, Ynew, ELIYXTT, 150, cmap='viridis', vmax=np.amax(ELIYXTT), vmin=np.amin(ELIYXTT))

# Remove gridlines, axis, and background
ax.grid(False)
ax.set_axis_off()

# Animation function to update view angle (top-down and rotating around y-axis)
def update(frame):
    ax.view_init(elev=frame % 360, azim=0)  # Elevation fixed (side view), changing azimuth to simulate left-to-right flipping
    return fig,

# Create animation
frames = 360  # Full rotation over 360 frames
anim = FuncAnimation(fig, update, frames=frames, interval=1000/120)

# Save as GIF
anim.save('spinning_donut.gif', writer=PillowWriter(fps=60))

plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

# Basic visualisation of the data

#### Top down

In [ ]:
# Creates a top down 2D plot

# Start timing
start_time = time.time()

# Create a figure and axis object
fig, ax = plt.subplots()

# Create a filled contour plot
contour = ax.contourf(Xnew, Ynew, ELIYXTT, 256, vmax=np.amax(ELIYXTT.max()), vmin=np.amin(ELIYXTT.min()))

# Set the title
# ax.set_title('Corrected Data')

# Set axis to be equal
ax.set_aspect('equal')

# Add a colorbar
cbar = fig.colorbar(contour, ax=ax)
cbar.set_label('Z Position / um', fontsize = fsize)

# Use MaxNLocator to determine suitable ticks
cbar.locator = MaxNLocator(nbins=10)  # Increase the number of bins for more ticks
cbar.update_ticks()

cbar.ax.tick_params(labelsize = fsize) 

# Alternatively, you can set custom ticks manually
# cbar.set_ticks(np.linspace(np.amin(ELIYXTT.min()), np.amax(ELIYXTT.max()), num=15))

# Set color limits (again, this may be redundant but included for consistency)
contour.set_clim(vmax=np.amax(ELIYXTT.max()), vmin=np.amin(ELIYXTT.min()))

# Label the axes
ax.set_xlabel('X Position / mm', fontsize = fsize)
plt.xticks(fontsize = fsize)                  
ax.set_ylabel('Y Position / mm', fontsize = fsize)
plt.yticks(fontsize = fsize)                  

# Remove the top and right spines
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Adjust the layout to avoid overlap
fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

# Resize the plot area
fig.set_size_inches(160/25.4, 160/25.4)

output = file_name.replace('.xyz', '_circle.png')
fig.savefig(output, dpi=150)

# Show the plot
plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

#### Using a scale bar instead of tick marks

In [ ]:
# Start timing
start_time = time.time()

# Create a figure and axis object
fig, ax = plt.subplots()

# Create a filled contour plot
contour = ax.contourf(Xnew, Ynew, ELIYXTT, 256, vmax=np.amax(ELIYXTT.max()), vmin=np.amin(ELIYXTT.min()))

# Set axis to be equal
ax.set_aspect('equal')

# Add a colorbar
cbar = fig.colorbar(contour, ax=ax, shrink=0.85)
cbar.set_label('Z Position / um', fontsize=fsize)

# Use MaxNLocator to determine suitable ticks
cbar.locator = MaxNLocator(nbins=10)
cbar.update_ticks()

cbar.ax.tick_params(labelsize=fsize)

# Set color limits
contour.set_clim(vmax=np.amax(ELIYXTT.max()), vmin=np.amin(ELIYXTT.min()))

# Turn off ticks and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('')

# Remove all spines
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)

# Add a scale bar of 10mm in the bottom left corner
scalebar_length = 5 #mm
x_pos = -6
y_pos = -7
ax.plot([x_pos, x_pos + scalebar_length], [y_pos, y_pos], color='black', lw=10)  # Thicker scale bar
ax.text(x_pos, y_pos - 0.02 * (ax.get_ylim()[1] - ax.get_ylim()[0]), '5 mm', fontsize=fsize, va='top')

# Resize the plot area
fig.set_size_inches(160/25.4, 160/25.4)

output = file_name.replace('.xyz', '_circle_scalebar.png')
fig.savefig(output, dpi=150)

# Show the plot
plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

#### Side on

In [ ]:
# Creates a side on 2D plot

# Start timing
start_time = time.time()

# Generate X values as indices
x_values = np.arange(len(z_clean))

# Create the plot
plt.figure(figsize=(160/25.4, 80/25.4))
plt.scatter(x_values, z_clean, alpha = 0.05, s = 2)
plt.xlabel('Index')
plt.ylabel('Height/μm')
output = file_name.replace('.xyz', '_2D.png')


# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.savefig('2ndtilt_slice', dpi=150)

plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

#### 3D (not currently saved as image)

In [ ]:
# Creates a fun 3D plot

# Start timing
start_time = time.time()

# Create a figure and an axes for 3D plotting
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Create the 3D contour plot
contour = ax.contour3D(Xnew, Ynew, ELIYXTT, 150, cmap='viridis', vmax=np.amax(ELIYXTT), vmin=np.amin(ELIYXTT))

# Labels and title
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('ELIYXTT')

# Show the plot
plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

# This code quantifies surface roughness

#### This code finds the Root mean square of the surface

In [ ]:
# Calculate the mean
mean_value = np.mean(z_clean)
print(f"Mean of the array: {mean_value:.4g} µm")

# Calculate squared differences from the mean
squared_differences = np.square(z_clean - mean_value)
print("Squared differences from the mean:", squared_differences)

# Calculate the mean of the squared differences
mean_of_squared_differences = np.mean(squared_differences)
print(f"Mean of the squared differences: {mean_of_squared_differences:.4g} µm²")

# Calculate the RMS value
rms_value = np.sqrt(mean_of_squared_differences)
print(f"The RMS value of the 2D array is: {rms_value:.4g} µm")

#### This code uses top 5 and bottom 5 % of datapoints

In [ ]:
# Calculate the percentage for top 5% and bottom 5%
percentile_cutoff = 0.05

# Start timing
start_time = time.time()

sorted_values = np.sort(z_clean)

# Shift the data so that the mean is at 0
shifted_values = sorted_values

# Recalculate cumulative counts
cumulative_counts = np.arange(1, len(shifted_values) + 1)
# Convert counts to normalized percentages
percentage_below = cumulative_counts / len(shifted_values)

# Calculate the number of data points that corresponds to 5% of the total data
n_points = len(shifted_values)
cutoff_count = int(n_points * percentile_cutoff)

# Select the bottom 5% and top 5% points
bottom_5_percent_values = shifted_values[:cutoff_count]
top_5_percent_values = shifted_values[-cutoff_count:]

# Find the point closest to the mean (0) for the bottom 5% and top 5%
closest_bottom_5_percent = bottom_5_percent_values[np.abs(bottom_5_percent_values).argmin()]
closest_top_5_percent = top_5_percent_values[np.abs(top_5_percent_values).argmin()]

# Calculate the height value at the 5% cutoff
height_at_bottom_5_percent = shifted_values[cutoff_count - 1]
height_at_top_5_percent = shifted_values[-cutoff_count]

#Finding the sample area and galled area
SA = math.pi*(6.35**2 - 3.1875**2) #From ASTM G196 sample drawing

# Plot the cumulative frequency distribution
plt.figure(figsize=(12, 6))
plt.plot(percentage_below, shifted_values)

# Plot the point closest to the mean for the bottom 5% and top 5%
plt.plot(percentage_below[np.searchsorted(shifted_values, closest_bottom_5_percent)],
         closest_bottom_5_percent, 'ro', label=f'Closest Bottom 5% ({closest_bottom_5_percent:.2f})')

plt.plot(percentage_below[np.searchsorted(shifted_values, closest_top_5_percent)],
         closest_top_5_percent, 'bo', label=f'Closest Top 5% ({closest_top_5_percent:.2f})')

# Plot the shifted mean point (now zero)
plt.plot(percentage_below[np.searchsorted(shifted_values, 0)], 0, 'go', label=f'Zero-plane and mean (0)')

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Cumulative Frequency')
plt.ylabel('Height/um')
plt.legend(frameon=0)
plt.tight_layout
plt.savefig('points5%.png', dpi = 150)
plt.show()

# Volumes in mm^3 for 5% data points
volumes_below_lower_bound = np.sum(bottom_5_percent_values/1000 * pixel_size)
volumes_above_upper_bound = np.sum(top_5_percent_values/1000 * pixel_size)

# Calculate number of points and percentages
points_below_lower_bound = len(bottom_5_percent_values)
points_above_upper_bound = len(top_5_percent_values)

percentage_bottom_5_percent = points_below_lower_bound / len(shifted_values) * 100
percentage_top_5_percent = points_above_upper_bound / len(shifted_values) * 100

# Print the results
print(f'Bottom 5% points, area of {points_below_lower_bound * pixel_size:.4g} mm² ({percentage_bottom_5_percent:.2f}% of points)')
print(f'Top 5% points, area of {points_above_upper_bound * pixel_size:.4g} mm² ({percentage_top_5_percent:.2f}% of points)')
print('\n')
# Print percentages relative to the sample area
print(f'Using calculated area of sample to be {SA:.4g} mm²')
print(f'Below lower bound: {points_below_lower_bound * pixel_size / SA * 100:.4g}%')
print(f'Above upper bound: {points_above_upper_bound * pixel_size / SA * 100:.4g}%')
print('\n')
# Print volumn of peak and trough in mm^3
print(f'Trough volume of {abs(volumes_below_lower_bound):.4g} mm³')
print(f'Peak volume of {volumes_above_upper_bound:.4g} mm³')
print(f'Total galled volume of {abs(volumes_below_lower_bound) + volumes_above_upper_bound:.4g} mm³')
print('\n')
# Print the height values at 5% cutoff
print(f'Height at 5% cutoff (bottom): {height_at_bottom_5_percent:.4g} µm')
print(f'Height at 5% cutoff (top): {height_at_top_5_percent:.4g} µm')
print(f'Range of the data {height_at_top_5_percent - height_at_bottom_5_percent:.4g} µm')
print('\n')

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

#### This version uses 5% by range so anything below or above the 5% mark

In [ ]:
# Start timing
start_time = time.time()

sorted_values = np.sort(z_clean)

# Shift the data so that the mean is at 0
shifted_values = sorted_values

# Recalculate cumulative counts
cumulative_counts = np.arange(1, len(shifted_values) + 1)
# Convert counts to normalized percentages
percentage_below = cumulative_counts / len(shifted_values)

# Calculate range and determine the top and bottom 5%
data_range = np.max(shifted_values) - np.min(shifted_values)
lower_bound = np.min(shifted_values) + 0.05 * data_range
upper_bound = np.max(shifted_values) - 0.05 * data_range

# Filter values outside the acceptable range
below_lower_bound_values = shifted_values[shifted_values < lower_bound]
above_upper_bound_values = shifted_values[shifted_values > upper_bound]

# Calculate how many points are outside the acceptable range
points_below_lower_bound = len(below_lower_bound_values)
points_above_upper_bound = len(above_upper_bound_values)

percentage_below_lower_bound = points_below_lower_bound / len(shifted_values) * 100
percentage_above_upper_bound = points_above_upper_bound / len(shifted_values) * 100

# Volume calculation for points in mm^3
volumes_below_lower_bound = np.sum(below_lower_bound_values / 1000 * pixel_size)
volumes_above_upper_bound = np.sum(above_upper_bound_values / 1000 * pixel_size)

# Finding the sample area and galled area
SA = math.pi * (6.35**2 - 3.1875**2)  # From ASTM G196 sample drawing

# Plot the cumulative frequency distribution
plt.figure(figsize=(12, 6))
plt.plot(percentage_below, shifted_values)

# Plot the values outside the acceptable range
plt.plot(percentage_below[np.searchsorted(shifted_values, lower_bound)], lower_bound, 'ro', label=f'Lower Bound ({lower_bound:.2f})')
plt.plot(percentage_below[np.searchsorted(shifted_values, upper_bound)], upper_bound, 'bo', label=f'Upper Bound ({upper_bound:.2f})')

# Plot the shifted mean point (now zero)
plt.plot(percentage_below[np.searchsorted(shifted_values, 0)], 0, 'go', label='Zero-plane and mean (0)')

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Cumulative Frequency')
plt.ylabel('Height/um')
plt.legend(frameon=0)
plt.tight_layout()
plt.savefig('range5%.png', dpi=150)
plt.show()

# Print the results
print(f'Below lower bound: {points_below_lower_bound} points, with an area of {points_below_lower_bound * pixel_size:.4g} mm² ({percentage_below_lower_bound:.2f}% of points)')
print(f'Above upper bound: {points_above_upper_bound} points, with an area of {points_above_upper_bound * pixel_size:.4g} mm² ({percentage_above_upper_bound:.2f}% of points)')
print('\n')
# Print percentages relative to the sample area
print(f'Using calculated area of sample to be {SA:.4g} mm²')
print(f'Below lower bound: {points_below_lower_bound * pixel_size / SA * 100:.4g}%')
print(f'Above upper bound: {points_above_upper_bound * pixel_size / SA * 100:.4g}%')
print('\n')
# Print volume of peak and trough in mm^3
print(f'Trough volume of {abs(volumes_below_lower_bound):.4g} mm³')
print(f'Peak volume of {volumes_above_upper_bound:.4g} mm³')
print(f'Total galled volume of {abs(volumes_below_lower_bound) + volumes_above_upper_bound:.4g} mm³')
print('\n')

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")


#### This code uses threshold from mean

In [ ]:
# Set a threshold range around the mean (which is now 0)
threshold = 6  # Adjust this value to your needs

# Start timing
start_time = time.time()

sorted_values = np.sort(z_clean)

# Calculate the mean
mean_value = sorted_values.mean()

# Shift the data so that the mean is at 0
shifted_values = sorted_values - mean_value

# Recalculate cumulative counts
cumulative_counts = np.arange(1, len(shifted_values) + 1)
# Convert counts to normalized percentages
percentage_below = cumulative_counts / len(shifted_values)

lower_bound = -threshold
upper_bound = threshold

# --- The range is picked -----------------------------------------------------------------------------------------------------------------------------------------------------

# Filter values outside the acceptable range
below_lower_bound_values = shifted_values[shifted_values < lower_bound]
above_upper_bound_values = shifted_values[shifted_values > upper_bound]

# Calculate how many points are outside the acceptable range
points_below_lower_bound = len(below_lower_bound_values)
points_above_upper_bound = len(above_upper_bound_values)

percentage_below_lower_bound = points_below_lower_bound / len(shifted_values) * 100
percentage_above_upper_bound = points_above_upper_bound / len(shifted_values) * 100

# Volume calculation for points in mm^3
volumes_below_lower_bound = np.sum(below_lower_bound_values/1000 * pixel_size)
volumes_above_upper_bound = np.sum(above_upper_bound_values/1000 * pixel_size)

#Finding the sample area and galled area
SA = math.pi*(6.35**2 - 3.1875**2) #From ASTM G196 sample drawing

# Plot the cumulative frequency distribution
plt.figure(figsize=(12, 6))
plt.plot(percentage_below, shifted_values)

# Plot the values outside the acceptable range
plt.plot(percentage_below[np.searchsorted(shifted_values, lower_bound)], lower_bound, 'ro', label=f'Lower Bound ({lower_bound:.2f})')
plt.plot(percentage_below[np.searchsorted(shifted_values, upper_bound)], upper_bound, 'bo', label=f'Upper Bound ({upper_bound:.2f})')

# Plot the shifted mean point (now zero)
plt.plot(percentage_below[np.searchsorted(shifted_values, 0)], 0, 'go', label=f'Zero-plane and mean (0)')

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Cumulative Frequency')
plt.ylabel('Height/um')
plt.legend(frameon=0)
plt.tight_layout
plt.savefig('mean.png', dpi = 150)
plt.show()

# Print the results
print(f'Below lower bound: {points_below_lower_bound} points, with an area of {points_below_lower_bound * pixel_size:.4g} mm² ({percentage_below_lower_bound:.2f}% of points)')
print(f'Above upper bound: {points_above_upper_bound} points, with an area of {points_above_upper_bound * pixel_size:.4g} mm² ({percentage_above_upper_bound:.2f}% of points)')
print('\n')
# Print percentages relative to the sample area
print(f'Using calculated area of sample to be {SA:.4g} mm²')
print(f'Below lower bound: {points_below_lower_bound * pixel_size / SA * 100:.4g}%')
print(f'Above upper bound: {points_above_upper_bound * pixel_size / SA * 100:.4g}%')
print('\n')
# Print volumn of peak and trough in mm^3
print(f'Trough volume of {abs(volumes_below_lower_bound):.4g} mm³')
print(f'Peak volume of {volumes_above_upper_bound:.4g} mm³')
print(f'Total galled volume of {abs(volumes_below_lower_bound) + volumes_above_upper_bound:.4g} mm³')
print('\n')

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

#### This code uses threshold from median (zero-plane)

In [ ]:
# Set a threshold range around the median
threshold = 6  # Adjust this value to your needs

# Start timing
start_time = time.time()

sorted_values = np.sort(z_clean)

# Recalculate cumulative counts
cumulative_counts = np.arange(1, len(sorted_values) + 1)
# Convert counts to normalized percentages
percentage_below = cumulative_counts / len(sorted_values)

lower_bound = -threshold
upper_bound = threshold

# --- The range is picked -----------------------------------------------------------------------------------------------------------------------------------------------------

# Filter values outside the acceptable range
below_lower_bound_values = shifted_values[sorted_values < lower_bound]
above_upper_bound_values = shifted_values[sorted_values > upper_bound]

# Calculate how many points are outside the acceptable range
points_below_lower_bound = len(below_lower_bound_values)
points_above_upper_bound = len(above_upper_bound_values)

percentage_below_lower_bound = points_below_lower_bound / len(sorted_values) * 100
percentage_above_upper_bound = points_above_upper_bound / len(sorted_values) * 100

# Volume calculation for points in mm^3
volumes_below_lower_bound = np.sum(below_lower_bound_values/1000 * pixel_size)
volumes_above_upper_bound = np.sum(above_upper_bound_values/1000 * pixel_size)

#Finding the sample area and galled area
SA = math.pi*(6.35**2 - 3.1875**2) #From ASTM G196 sample drawing


# Plot the cumulative frequency distribution
plt.figure(figsize=(12, 6))
plt.plot(percentage_below, sorted_values)

# Plot the values outside the acceptable range
plt.plot(percentage_below[np.searchsorted(sorted_values, lower_bound)], lower_bound, 'ro', label=f'Lower Bound ({lower_bound:.2f})')
plt.plot(percentage_below[np.searchsorted(sorted_values, upper_bound)], upper_bound, 'bo', label=f'Upper Bound ({upper_bound:.2f})')

# Plot the shifted mean point (now zero)
plt.plot(percentage_below[np.searchsorted(sorted_values, 0)], 0, 'go', label=f'Zero plane (0)')

# Remove top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add labels and title
plt.xlabel('Cumulative Frequency')
plt.ylabel('Height/um')
plt.legend(frameon=0)
plt.tight_layout
plt.savefig('median.png', dpi = 150)
plt.show()

# Print the results
print(f'Below lower bound: {points_below_lower_bound} points, with an area of {points_below_lower_bound * pixel_size:.4g} mm² ({percentage_below_lower_bound:.2f}% of points)')
print(f'Above upper bound: {points_above_upper_bound} points, with an area of {points_above_upper_bound * pixel_size:.4g} mm² ({percentage_above_upper_bound:.2f}% of points)')
print('\n')
# Print percentages relative to the sample area
print(f'Using calculated area of sample to be {SA:.4g} mm²')
print(f'Below lower bound: {points_below_lower_bound * pixel_size / SA * 100:.4g}%')
print(f'Above upper bound: {points_above_upper_bound * pixel_size / SA * 100:.4g}%')
print('\n')
# Print volumn of peak and trough in mm^3
print(f'Trough volume of {abs(volumes_below_lower_bound):.4g} mm³')
print(f'Peak volume of {volumes_above_upper_bound:.4g} mm³')
print(f'Total galled volume of {abs(volumes_below_lower_bound) + volumes_above_upper_bound:.4g} mm³')
print('\n')
# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

# This code helps visualize tilt correction

In [ ]:
# Start timing
start_time = time.time()

data = dfnew.to_numpy()

# Flatten the array to get all Z values
z_values = data.flatten()

# Generate X values as indices
x_values = np.arange(len(z_values))

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(x_values, z_values, alpha = 0.05, s = 2, color = "tab:orange")
plt.xlabel('Index')
plt.ylabel('Height/μm')
output = file_name.replace('.xyz', '_2D.png')
plt.savefig('raw_Data.png', dpi=150)
plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

In [ ]:
# Start timing
start_time = time.time()

data = ELIX2.to_numpy()

# Flatten the array to get all Z values
z_values = data.flatten()

# Generate X values as indices
x_values = np.arange(len(z_values))

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(x_values, z_values, alpha = 0.05, s = 2, color = "tab:green")
plt.xlabel('Index')
plt.ylabel('Height/μm')
output = file_name.replace('.xyz', '_2D.png')
plt.savefig('1st_tilt.png', dpi=150)
plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

In [ ]:
start_time = time.time()

# Example of how you might have multiple dataframes (replace this with your actual setup)
dataframes = [dfnew, ELIX2, ELIYXTT]
labels = ['dfnew', 'ELIX2', 'ELIYXTT']
colors = ['tab:orange', 'tab:green', 'tab:blue']

plt.figure(figsize=(160/25.4, 80/25.4))

# Iterate over each dataframe
for idx, df in enumerate(dataframes):
    # Convert dataframe to numpy array outside the loop
    data = df.to_numpy()
    
    # Flatten the array to get all Z values
    z_values = data.flatten()

    # Downsample the data to speed up plotting, e.g., plot every 10th point
    if idx == 0:
        step = 2
    else:
        step = 10  # Change step size based on your dataset
    
    x_values = np.arange(len(z_values))[::step]
    z_values = z_values[::step]

    # Create the plot for each dataframe
    plt.scatter(x_values, z_values, alpha=0.2, s=1, label=labels[idx], color=colors[idx])

plt.xlabel('Index')
plt.ylabel('Z')
legend = plt.legend(loc= 1,frameon=False)

for handle in legend.legendHandles:
    handle.set_alpha(1)

# Remove the top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Save the figure
plt.savefig('allstakced.png', dpi=150)

plt.show()

# End timing for the whole process
end_time = time.time()

# Print the timing results
print(f"Total execution time: {end_time - start_time:.2f} seconds")

# This is Sams old galling area code, im keeping here because its good to check the code outputs the same numbers

In [ ]:
#Finding the galled area and displaced volumes using the Cumulative Frequency Distribution
Ra = 6 #Ra in um

Zsortnan = np.sort(np.ndarray.flatten(np.asarray(ELIYXTT)))
Zsort = Zsortnan[~np.isnan(Zsortnan)]

#Size of each pixel for corrected sample
Xnewsize = 0.003484/(XTilt*XTilt2)
Ynewsize = 0.003484/(YTilt*YTilt2)

for i in range(0, len(Zsort), 1):
    if Zsort[i]>-Ra:
        ineg = i
        print('ineg: ', ineg)
        break
    else:
        continue
    
    
for i in range(ineg, len(Zsort), 1):
    if Zsort[i]>Ra:
        ipos = i
        print('ipos: ', ipos)
        break
    else:
        continue

#Finding the sample area and galled area
SA = math.pi*(6.35**2 - 3.1875**2) #From ASTM G196 sample drawing

if 100*Xnewsize*Ynewsize*(ineg+(len(Zsort)-ipos))/SA <= 0:
    GA = 0
else:
    GA = 100*Xnewsize*Ynewsize*(ineg+(len(Zsort)-ipos))/SA

if ineg==0:
    DVneg = 0
else:
    DVneg = -np.sum(Zsort[0:ineg-1])*Xnewsize*Ynewsize/1000
    
DVpos = np.sum(Zsort[ipos:])*Xnewsize*Ynewsize/1000
DVtot = DVneg + DVpos

TGA = 100*Xnewsize*Ynewsize*(ineg)/SA
PGA = 100*Xnewsize*Ynewsize*((len(Zsort)-ipos))/SA

print('List length:',len(Zsort))
print('Trough Area / %: ', TGA)
print('Peak Area / %: ', PGA)
print('Galled Area / %: ', GA)
print()
print('Trough Volume (mm3): ', DVneg)
print('Peak Volume (mm3): ', DVpos)
print('Total Galled Volume (mm3): ', DVtot)
print()
print('Xszie,Ysize:',Xnewsize*Ynewsize)
print('Pixel size:',pixel_size)
print('surafce area:',SA)